In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
# Inisialisasi splitter untuk memisahkan dokumen
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)

# Inisialisasi variabel untuk menampung dokumen yang digabung
combined_docs = []

# Fungsi untuk memuat dan memisahkan dokumen
def load_and_split_pdf(file_path, text_splitter):
    loader = PyPDFLoader(file_path)
    data = loader.load()  # Muat dokumen sebagai satu kesatuan
    split_docs = text_splitter.split_documents(data)  # Pisahkan dokumen
    return split_docs

# Daftar file yang akan diproses
file_paths = [
    "5. UU-40-2007 PERSEROAN TERBATAS.pdf",
    "e39ab-uu-nomor-8-tahun-1999.pdf",
    "kolonial_kuh_perdata_fix.pdf",
    "KUH DAGANG.pdf",
    "UU Nomor  19 Tahun 2016.pdf",
    "UU Nomor 13 Tahun 2003.pdf",
    "UU_1999_30.pdf",
    "UU_Nomor_11_Tahun_2020-compressed.pdf",
]

# Proses setiap dokumen secara otomatis
for file_path in file_paths:
    print(f"Processing: {file_path}")
    additional_docs = load_and_split_pdf(file_path, text_splitter)  # Load dan split dokumen
    combined_docs += additional_docs  # Gabungkan ke dalam combined_docs
    print(f"Added {len(additional_docs)} documents from {file_path}")

# Total dokumen setelah penggabungan
print("Total number of combined documents: ", len(combined_docs))

Processing: 5. UU-40-2007 PERSEROAN TERBATAS.pdf
Added 280 documents from 5. UU-40-2007 PERSEROAN TERBATAS.pdf
Processing: e39ab-uu-nomor-8-tahun-1999.pdf
Added 87 documents from e39ab-uu-nomor-8-tahun-1999.pdf
Processing: kolonial_kuh_perdata_fix.pdf
Added 1150 documents from kolonial_kuh_perdata_fix.pdf
Processing: KUH DAGANG.pdf
Added 604 documents from KUH DAGANG.pdf
Processing: UU Nomor  19 Tahun 2016.pdf
Added 44 documents from UU Nomor  19 Tahun 2016.pdf
Processing: UU Nomor 13 Tahun 2003.pdf
Added 242 documents from UU Nomor 13 Tahun 2003.pdf
Processing: UU_1999_30.pdf
Added 84 documents from UU_1999_30.pdf
Processing: UU_Nomor_11_Tahun_2020-compressed.pdf
Added 2090 documents from UU_Nomor_11_Tahun_2020-compressed.pdf
Total number of combined documents:  4581


In [6]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
load_dotenv() 


True

In [27]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.2,               # Sesuai dengan generation_config
    top_p=0.95,                    # Menambahkan top_p
    top_k=40,                      # Menambahkan top_k
    max_tokens=8192,               # Menyesuaikan max_output_tokens
    response_mime_type="text/plain" # Menyesuaikan response_mime_type
)


In [ ]:
# Define system prompt with context placeholder
system_prompt = """
Anda adalah AI hukum yang sangat terlatih, dirancang untuk menjawab pertanyaan pengguna dengan cara yang jelas, akurat, dan ramah. Anda bertugas untuk:

1. **Pemanfaatan Konteks Dokumen**:
   - Berikut adalah konteks dokumen yang ditemukan:
     {context}
   - Gunakan konteks ini untuk mendukung jawaban Anda jika relevan dengan pertanyaan pengguna.
   - Jika konteks dokumen tidak relevan dengan pertanyaan, abaikan konteks tersebut dan gunakan pengetahuan hukum Anda untuk memberikan jawaban.

2. **Fokus pada Pertanyaan**:
   - Berikan jawaban yang langsung menjawab inti pertanyaan pengguna.
   - Jangan menyertakan informasi yang tidak relevan atau tidak mendukung jawaban.

3. **Penyampaian Jawaban**:
   - Jawaban Anda harus terstruktur dengan format berikut:
     - **Jawaban Langsung**: Jawab inti pertanyaan dengan ringkas dan jelas.
     - **Penjelasan**: Berikan alasan atau dasar hukum yang mendukung jawaban Anda.
     - Jika ada pasal yang relevan, tambahkan rujukan pasal tersebut. Jika tidak ada, cukup berikan jawaban tanpa menyebutkan bahwa rujukan tidak ditemukan.

4. **Nada dan Gaya**:
   - Gunakan bahasa yang profesional tetapi tetap ramah dan mudah dipahami.
   - Hindari istilah teknis yang berlebihan, dan jika digunakan, berikan penjelasan singkat untuk membantu pengguna memahami.

5. **Ketentuan Khusus**:
   - Jika retrieval menghasilkan dokumen yang tidak relevan, berikan jawaban berdasarkan pengetahuan hukum Anda tanpa menyebutkan informasi retrieval yang tidak relevan.
   - Jangan memberikan informasi yang tidak diminta atau spekulatif.

**Catatan Penting**:
- Prioritaskan keakuratan dan relevansi dalam setiap jawaban.
- Jangan menyertakan bagian "Rujukan Pasal" jika tidak ada pasal yang relevan.
- Fokus utama Anda adalah memberikan jawaban langsung yang akurat, relevan, dan mendukung kebutuhan pengguna.

"""

In [32]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv



load_dotenv() 

user_input = 'Apa itu perjanjian jual beli?'

vectorstore = Chroma.from_documents(documents=combined_docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

retrieved_docs = retriever.invoke(user_input)


llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.2,               # Sesuai dengan generation_config
    top_p=0.95,                    # Menambahkan top_p
    top_k=40,                      # Menambahkan top_k
    max_tokens=8192,               # Menyesuaikan max_output_tokens
    response_mime_type="text/plain" # Menyesuaikan response_mime_type
)



prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(
    llm,
    prompt,
)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": user_input})
print(response["answer"])


**Jawaban Langsung:** Perjanjian jual beli adalah suatu persetujuan di mana pihak yang satu mengikatkan dirinya untuk menyerahkan suatu barang, dan pihak yang lain untuk membayar harga yang telah dijanjikan.

**Penjelasan:**  Jual beli merupakan suatu bentuk perjanjian timbal balik.  Penjual memiliki kewajiban untuk menyerahkan barang yang dijual, sedangkan pembeli berkewajiban untuk membayar harga barang tersebut sesuai dengan kesepakatan. Kesepakatan tersebut dapat terjadi secara lisan maupun tertulis.

**Rujukan Pasal:** Pasal 1457 KUHPerdata.


In [35]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = '''Anda adalah asisten hukum AI yang dirancang untuk memahami dan mengolah pertanyaan hukum. Tugas Anda adalah:

1. **Mengidentifikasi Kebutuhan Konteks**:
   - Berdasarkan sejarah percakapan (chat history) dan dokumen hukum yang relevan, tentukan apakah pertanyaan terbaru pengguna memerlukan konteks tambahan untuk dipahami sepenuhnya.

2. **Reformulasi Pertanyaan**:
   - Jika pertanyaan pengguna merujuk pada konteks dari percakapan sebelumnya, reformulasikan pertanyaan tersebut menjadi pertanyaan mandiri yang jelas dan lengkap tanpa memerlukan chat history.
   - Jika pertanyaan sudah mandiri dan jelas, kembalikan pertanyaan tersebut apa adanya.

3. **Tujuan Reformulasi**:
   - Buat pertanyaan mandiri agar sistem dapat mencari dokumen hukum yang relevan dan memberikan jawaban yang akurat.

**Peraturan**:
- Jangan menjawab pertanyaan.
- Jangan memberikan penjelasan tambahan.
- Hanya kembalikan pertanyaan dalam format yang mandiri dan dapat dipahami tanpa referensi ke chat history.
- Jika pertanyaan sudah jelas dan tidak memerlukan perubahan, kembalikan seperti apa adanya.
'''


contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)



In [36]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [37]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "Apa itu perjanjian jual beli menurut KUHPerdata?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "Apa kewajiban penjual dalam perjanjian jual beli tersebut?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

**Jawaban Langsung:** Penjual memiliki dua kewajiban utama, yaitu menyerahkan barang dan menanggung barang yang dijualnya.

**Penjelasan:** Kewajiban menyerahkan barang berarti penjual wajib memindahkan barang yang dijual ke dalam kekuasaan dan hak milik pembeli.  Ini termasuk memastikan barang bebas dari cacat atau sengketa kepemilikan. Kewajiban menanggung barang berarti penjual bertanggung jawab atas tuntutan pihak ketiga atas barang tersebut, misalnya jika ada pihak lain yang mengklaim memiliki hak atas barang tersebut. Penjual juga wajib menanggung beban atas barang yang tidak diberitahukan kepada pembeli saat perjanjian jual beli.  Biaya penyerahan barang ditanggung oleh penjual, sedangkan biaya pengambilan barang ditanggung oleh pembeli, kecuali diperjanjikan lain.

**Rujukan Pasal:** Pasal 1474, 1475, 1476, dan 1492 KUHPerdata.


In [38]:
print(ai_msg_1["answer"])

**Jawaban Langsung:** Perjanjian jual beli adalah suatu persetujuan dengan mana pihak yang satu mengikatkan dirinya untuk menyerahkan suatu kebendaan, dan pihak yang lain untuk membayar harga yang telah dijanjikan.

**Penjelasan:**  KUH Perdata mendefinisikan jual beli sebagai suatu perjanjian timbal balik (sinalagmatis) di mana terdapat dua pihak yang saling berjanji. Pihak pertama, yaitu penjual, berjanji untuk menyerahkan suatu barang. Pihak kedua, yaitu pembeli, berjanji untuk membayar harga barang tersebut.  Perjanjian ini bersifat konsensual, artinya perjanjian jual beli sah terjadi saat tercapai kesepakatan antara penjual dan pembeli mengenai barang dan harganya, meskipun barang belum diserahkan dan harga belum dibayar.

**Rujukan Pasal:** Pasal 1457 KUHPerdata.
